In [258]:
import pandas as pd
import tensorflow as tf
import numpy as np
import imutils
import cv2 as cv
import matplotlib.pylab as plt
from sklearn.metrics import classification_report
from google.colab import files
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import img_to_array

In [259]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [260]:
train_path = '/content/drive/MyDrive/input/Car_Brand_Logos/Train'
                                                      
valid_path = '/content/drive/MyDrive/input/Car_Brand_Logos/Test'
test_path = '/content/drive/MyDrive/input/Test'
   
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   )

valid_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 color_mode='grayscale')
                                                
validation_set = valid_datagen.flow_from_directory(valid_path,
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                             color_mode='grayscale')
test_set = valid_datagen.flow_from_directory(test_path,
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                             color_mode='grayscale')

Found 2513 images belonging to 8 classes.
Found 400 images belonging to 8 classes.
Found 136 images belonging to 8 classes.


In [261]:
model = keras.Sequential([
     layers.Conv2D(32, 7,1, 'same',activation= 'relu', input_shape =(256,256,1) ),
     layers.Conv2D(32, 7,1, 'same',activation= 'relu'),
          layers.MaxPool2D(),
      layers.Conv2D(64, 7,1, 'same',activation= 'relu'),
      layers.Conv2D(64, 7,1, 'same',activation= 'relu'),
     layers.MaxPool2D(),
   layers.Conv2D(128, 7,1, 'same',activation= 'relu'),
      layers.Conv2D(128, 7,1, 'same',activation= 'relu'),
     layers.MaxPool2D(),
      layers.Conv2D(256, 7,1, 'same',activation= 'relu'),
      layers.Conv2D(256, 7,1, 'same',activation= 'relu'),
      layers.MaxPool2D(),
     layers.Conv2D(64, 5,1, 'same',activation= 'relu' ),
     layers.Conv2D(128, 5,1, 'same',activation= 'relu' ),
       layers.Conv2D(128, 5,1, 'same',activation= 'relu' ),
       layers.Conv2D(128, 5,1, 'same',activation= 'relu' ),
       layers.MaxPool2D(),
      layers.Conv2D(128, 5,1, 'same',activation= 'relu' ),
       layers.Conv2D(256, 5,1, 'same',activation= 'relu' ),
       layers.Conv2D(256, 5,1, 'same',activation= 'relu' ),   
     layers.Conv2D(64, 3,1, 'same',activation= 'relu' ),
    layers.Conv2D(64, 3,1, 'same',activation= 'relu' ),
    layers.MaxPool2D(),
    layers.Conv2D(128, 3,1, 'same',activation= 'relu' ),
    layers.Conv2D(128, 3,1, 'same',activation= 'relu' ),
     layers.MaxPool2D(),
    layers.Conv2D(256, 3,1, 'same',activation= 'relu' ),
    layers.Conv2D(256, 3,1, 'same',activation= 'relu' ),
    layers.MaxPool2D(),
    layers.Conv2D(512, 3,1, 'same',activation= 'relu' ),
    layers.Conv2D(512, 3,1, 'same',activation= 'relu' ),
    layers.Flatten(), 
     layers.BatchNormalization(),
    layers.Dense(50, 'relu'),
     layers.BatchNormalization(),
    layers.Dense(40, 'relu'),
    layers.Dense(8, activation = 'softmax')
])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_309 (Conv2D)         (None, 256, 256, 32)      1600      
                                                                 
 conv2d_310 (Conv2D)         (None, 256, 256, 32)      50208     
                                                                 
 max_pooling2d_105 (MaxPooli  (None, 128, 128, 32)     0         
 ng2D)                                                           
                                                                 
 conv2d_311 (Conv2D)         (None, 128, 128, 64)      100416    
                                                                 
 conv2d_312 (Conv2D)         (None, 128, 128, 64)      200768    
                                                                 
 max_pooling2d_106 (MaxPooli  (None, 64, 64, 64)       0         
 ng2D)                                               

In [262]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Epochdat',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)




model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy']
              )
history = model.fit(
  training_set,
  validation_data=validation_set,
  epochs=30,
  verbose=1,callbacks=[model_checkpoint_callback]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/30
75/79 [===========================>..] - ETA: 2s - loss: 2.0719 - accuracy: 0.1530

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


79/79 [==============================] - 57s 702ms/step - loss: 2.0729 - accuracy: 0.1496 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 2/30
79/79 [==============================] - 55s 700ms/step - loss: 2.0757 - accuracy: 0.1520 - val_loss: 2.0793 - val_accuracy: 0.1250
Epoch 3/30
79/79 [==============================] - 54s 687ms/step - loss: 2.0223 - accuracy: 0.1763 - val_loss: 2.0749 - val_accuracy: 0.1625
Epoch 4/30
79/79 [==============================] - 56s 705ms/step - loss: 1.9554 - accuracy: 0.2244 - val_loss: 2.0686 - val_accuracy: 0.2100
Epoch 5/30
79/79 [==============================] - 54s 679ms/step - loss: 1.8969 - accuracy: 0.2610 - val_loss: 2.0399 - val_accuracy: 0.2000
Epoch 6/30
79/79 [==============================] - 53s 666ms/step - loss: 1.8117 - accuracy: 0.3104 - val_loss: 2.0227 - val_accuracy: 0.1750
Epoch 7/30
79/79 [==============================] - 58s 735ms/step - loss: 1.7529 - accuracy: 0.3494 - val_loss: 1.8836 - val_accuracy: 0.2725
Epoch 8/30

In [263]:
model.save('/content/drive/MyDrive')

In [264]:

from sklearn.metrics import classification_report

In [266]:
test_labels=test_set.classes 
predictions=model.predict_generator(test_set, verbose='1')
y_pred = np.argmax(predictions, axis=-1)
print(classification_report(test_labels, y_pred))
                          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.24      0.24      0.24        17
           2       0.17      0.12      0.14        17
           3       0.12      0.12      0.12        17
           4       0.17      0.18      0.17        17
           5       0.18      0.18      0.18        17
           6       0.24      0.24      0.24        17
           7       0.18      0.24      0.21        17

    accuracy                           0.16       136
   macro avg       0.16      0.16      0.16       136
weighted avg       0.16      0.16      0.16       136



In [267]:
def slide_window(image,ws ):
  for y in range (0, image.shape[0]- ws[1],4):
    for x in range (0, image.shape[0]- ws[0],4):
      yield(x,y, image[y:y+ws[1], x:x+ws[0]])

In [268]:
def img_pyr(image, scale= 1.5, minSize=(128,128)) :
  yield image
  while True:
    w= int(image.shape[1]/scale)
    image = imutils.resize(image,width=w)
    if image.shape[0]<minSize[1] or image.shape[1]<minSize[0]:
      break
    yield image

In [269]:
scan_img = cv.imread('/content/drive/MyDrive/input/mercedes.jpg')
	
gray = cv.cvtColor(scan_img, cv.COLOR_BGR2GRAY)
gray = imutils.resize(gray, width=600)
INPUT_SIZE = (256, 256)
(H, W) = gray.shape[:2]
pyram=img_pyr(gray,1.5,(128,128))
rois = []
locs = []

In [270]:
for image in pyram:
  scale = W/ float(image.shape[1])
  for (x, y, roiOrig) in slide_window(image, (32,32)):
   x = int(x*scale)
   y = int(y*scale)
  w = int(128*scale)
  h = int(128*scale)
  roi = cv.resize(roiOrig, INPUT_SIZE)
  roi = img_to_array(roi)
  rois.append(roi)
  locs.append((x, y, x + w, y + h))

In [274]:
rois = np.array(rois, dtype="float32")
preds = model.predict(rois,32)
np.set_printoptions(threshold=np.inf)
logodict = {0:'Hyundai' ,1: 'Lexus', 2: 'Mazda' , 3: 'Mercedes', 4: 'Opel' , 5: 'Skoda', 6: 'Toyota', 7: 'Volkswagen'}
predicprob=[]
predicclass=[]
for i in preds:
 if i[np.argmax(i)]>0.5:
  predicprob.append([i[np.argmax(i)]*100])
  predicclass.append(np.argmax(i))
print("Logo:", logodict[predicclass[predicprob.index(max(predicprob))]]) 
print("Confidence%: ", max(predicprob)[0])

Logo: Opel
Confidence%:  100.0
